# 텐서플로우 설치 확인

In [ ]:
import tensorflow as tf
tf.__version__

# 데이터 불러오기

In [1]:
import numpy as np
import pandas as pd
import os

PATH = './data/'

train = pd.read_json(os.path.join(PATH, 'train.json'))
test = pd.read_json(os.path.join(PATH, 'test.json'))

# 데이터 들여다보기

In [ ]:
train.info()

# 데이터 그려보기

In [3]:
# (75, 75)의 형태로 변경해줍니다.
band_1 = np.array([np.array(band).reshape(75, 75) for band in train['band_1']])
band_2 = np.array([np.array(band).reshape(75, 75) for band in train['band_2']])

import matplotlib.pyplot as plt

# 실제로 레이블이 선박인 데이터
plt.figure(figsize=(7, 7))
ax = plt.subplot(2, 2, 1); ax.imshow(band_1[0])
ax.set_ylabel('ship', size = 15)
ax2 = plt.subplot(2, 2, 2); ax2.imshow(band_2[0])

# 실제로 레이블이 빙산인 데이터
ax = plt.subplot(2, 2, 3); ax.imshow(band_1[6])
ax.set_ylabel('iceberg', size = 15)
ax2 = plt.subplot(2, 2, 4); ax2.imshow(band_2[6])
plt.show()

<Figure size 700x700 with 4 Axes>

# 데이터 전처리 과정

In [4]:
def get_mean_std(array):
    return np.mean(array), np.std(array)

band_1_mean, band_1_std = get_mean_std(band_1)
band_2_mean, band_2_std = get_mean_std(band_2)

# 각 이미지별로 표준화를 적용합니다.
band_1_st = (band_1 - band_1_mean) / band_1_std
band_2_st = (band_2 - band_2_mean) / band_2_std

# 데이터 특성 및 검증 데이터 만들기

In [17]:
x_train = np.concatenate([band_1_st[:, :, :, np.newaxis], band_2_st[:, :, :, np.newaxis],
                          ((band_1_st+band_2_st)/2)[:, :, :, np.newaxis]], axis=-1)
y_train = np.array(train['is_iceberg'])

from sklearn.model_selection import train_test_split

# 훈련/테스트 데이터를 0.7/0.3의 비율로 분리합니다.
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state = 777)
print('훈련 데이터 {} 레이블 {}'.format(str(x_train.shape), str(y_train.shape)))
print('검증 데이터 {} 레이블 {}'.format(str(x_val.shape), str(y_val.shape)))

훈련 데이터 (1283, 75, 75, 3) 레이블 (1283,)
검증 데이터 (321, 75, 75, 3) 레이블 (321,)


# 모델 구성하기

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

model = Sequential()
# 입력 데이터의 형태를 꼭 명시해야 합니다.
model.add(Flatten(input_shape = (75, 75, 3))) # (75, 75, 3) -> (75 * 75 * 3)
model.add(Dense(512, activation = 'relu')) # 512개의 출력을 가지는 Dense 층
model.add(Dense(256, activation = 'relu')) # 256개의 출력을 가지는 Dense 층
model.add(Dense(128, activation = 'relu')) # 128개의 출력을 가지는 Dense 층
model.add(Dense(64, activation = 'relu')) # 64개의 출력을 가지는 Dense 층
model.add(Dense(1, activation = 'sigmoid')) # 1개의 출력을 가지는 신경망

model.compile(optimizer='adam', # 옵티마이저 : Adam
              loss = 'binary_crossentropy', # 손실 함수 : categorical_crossentropy
              metrics=['acc']) # 모니터링 할 평가지표 : acc

# 모델 학습하기

In [18]:
model.fit(x_train, y_train, 
                epochs = 10, 
                batch_size = 64, 
                validation_data = (x_val, y_val))

Train on 1283 samples, validate on 321 samples
Epoch 1/10
1283/1283 [==============================] - 27s 21ms/sample - loss: 1.6032 - acc: 0.5737 - val_loss: 1.8482 - val_acc: 0.5327
Epoch 2/10
1283/1283 [==============================] - 1s 505us/sample - loss: 1.4143 - acc: 0.6087 - val_loss: 1.5596 - val_acc: 0.6480
Epoch 3/10
1283/1283 [==============================] - 1s 567us/sample - loss: 0.8341 - acc: 0.6968 - val_loss: 1.2244 - val_acc: 0.6604
Epoch 4/10
1283/1283 [==============================] - 1s 533us/sample - loss: 1.0046 - acc: 0.7030 - val_loss: 0.7238 - val_acc: 0.6729
Epoch 5/10
1283/1283 [==============================] - 1s 498us/sample - loss: 0.6525 - acc: 0.7529 - val_loss: 0.7276 - val_acc: 0.6137
Epoch 6/10
1283/1283 [==============================] - 1s 571us/sample - loss: 0.4586 - acc: 0.8028 - val_loss: 0.6130 - val_acc: 0.6978
Epoch 7/10
1283/1283 [==============================] - 0s 366us/sample - loss: 0.2600 - acc: 0.8823 - val_loss: 0.7482 - val

# 제출하기

## 테스트 데이터셋 전처리 과정

In [ ]:
test_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
test_band_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])

test_band_1_st = (test_band_1 - band_1_mean) / band_1_std
test_band_2_st = (test_band_2 - band_2_mean) / band_2_std

x_test = np.concatenate([test_band_1_st[:, :, :, np.newaxis]
                          , test_band_2_st[:, :, :, np.newaxis]
                         , ((test_band_1_st+test_band_2_st)/2)[:, :, :, np.newaxis]], axis=-1)

## 예측하기 및 제출용 파일 만들기

In [ ]:
pred = model.predict(x_test)

# 제출용 파일을 만듭니다. 
submission = pd.DataFrame()
submission['id'] = test['id']
submission['is_iceberg'] = pred.reshape((pred.shape[0]))
submission.to_csv('sub.csv', index=False)